# Dependecies

In [ ]:
!pip install Boruta
from google.colab import files, drive
drive.mount('/content/drive')
!cp /content/drive/MyDrive/Studia/DS/Sem2/AML/Project2/boruta_py.py /usr/local/lib/python3.10/dist-packages/boruta/boruta_py.py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 1.1 MB/s eta 0:00:00
Mounted at /content/drive


In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from boruta import BorutaPy
import xgboost as xgb
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.metrics import make_scorer
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import accuracy_score
import time
import pandas as pd
import lightgbm as lgbm

# Data

In [ ]:
data_path = '/content/drive/MyDrive/Studia/DS/Sem2/AML/Project2/Data'

In [ ]:
X = np.loadtxt(data_path+'/x_train.txt')
y = np.loadtxt(data_path+'/y_train.txt')
test = np.loadtxt(data_path+'/x_test.txt')

# Choosing only a subset of features with boruta algorithm.

In [ ]:
rf = RandomForestClassifier(n_jobs=-1, class_weight=None, max_depth=7, random_state=0)
boruta = BorutaPy(rf, n_estimators='auto', verbose=2, random_state=0)

In [ ]:
boruta.fit(X, y)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	500
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	500
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	500
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	500
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	500
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	500
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	500
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	0
Tentative: 	23
Rejected: 	477
Iteration: 	9 / 100
Confirmed: 	16
Tentative: 	7
Rejected: 	477
Iteration: 	10 / 100
Confirmed: 	16
Tentative: 	7
Rejected: 	477
Iteration: 	11 / 100
Confirmed: 	16
Tentative: 	7
Rejected: 	477
Iteration: 	12 / 100
Confirmed: 	16
Tentative: 	4
Rejected: 	480
Iteration: 	13 / 100
Confirmed: 	16
Tentative: 	4
Rejected: 	480
Iteration: 	14 / 100
Confirmed: 	16
Tentative: 	4
Rejected: 	480
Iteration: 	15 / 100
Confirmed: 	16
Tentative: 	4
Rejected: 	480
Iteration: 	16 / 100
Confirmed: 	16
Tenta

BorutaPy(estimator=RandomForestClassifier(max_depth=7, n_estimators=85,
                                          n_jobs=-1,
                                          random_state=RandomState(MT19937) at 0x791E48509840),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x791E48509840, verbose=2)

In [ ]:
np.arange(X.shape[1])[boruta.ranking_ <= 25]

41

In [ ]:
np.save('/content/drive/MyDrive/Studia/DS/Sem2/AML/Project2/Boruta/support.npy', boruta.support_)
np.save('/content/drive/MyDrive/Studia/DS/Sem2/AML/Project2/Boruta/weak_support.npy', boruta.support_weak_)
np.save('/content/drive/MyDrive/Studia/DS/Sem2/AML/Project2/Boruta/ranking.npy', boruta.ranking_)

NameError: name 'boruta' is not defined

In [ ]:
boruta_support = np.load('/content/drive/MyDrive/Studia/DS/Sem2/AML/Project2/Boruta/support.npy')
boruta_support_weak = np.load('/content/drive/MyDrive/Studia/DS/Sem2/AML/Project2/Boruta/weak_support.npy')
boruta_support = boruta_support | boruta_support_weak

X_boruta = X[:, boruta_support]

# Money metric


In [ ]:
def profit_scoring(y_probs, y_true, n_features):

    y_true = y_true.ravel()
    y_probs = y_probs.ravel()
    y_pred = (y_probs > 0.5).astype(int)

    df = pd.DataFrame({'true': y_true, 'pred': y_pred, 'prob': y_probs})

    df = df.sort_values('prob', ascending=False)

    top_20_percent = df.head(int(len(df) * 0.2))

    correct_predictions = ((top_20_percent['pred'] == 1) & (top_20_percent['true'] == 1)).sum()
    profit = correct_predictions * 10 - n_features * 200 * (top_20_percent.shape[0]/1000)

    return profit

# XGB
Tunning hyperparameters with hyperopt

In [ ]:
def xgb_cv_profit_early_stop(estimator, X, y, n_folds=5):
  skf = StratifiedKFold(n_folds)
  profit = 0
  for cv_idx, (train_index, test_index) in enumerate(skf.split(X, y)):
    Xtrain, Xval, ytrain, yval = train_test_split(X[train_index, :], y[train_index], test_size=0.2, stratify=y[train_index])
    evaluation = [(Xval, yval)]
    estimator.fit(Xtrain, ytrain, eval_set=evaluation, verbose=False)
    probs = estimator.predict_proba(X[test_index, :])[:,1]
    profit += profit_scoring(probs, y[test_index], X.shape[1])
  profit /= n_folds
  return profit

In [ ]:
def xgb_cv_profit(estimator, X, y, n_folds=5):
  skf = StratifiedKFold(n_folds)
  profit = 0
  for cv_idx, (train_index, test_index) in enumerate(skf.split(X, y)):
    #evaluation = [(X[train_index, :], y[train_index]), (X[test_index, :], y[test_index])]
    estimator.fit(X[train_index, :], y[train_index], verbose=False)#, eval_set=evaluation, verbose=False, eval_metric='auc')
    probs = estimator.predict_proba(X[test_index, :])[:,1]
    profit += profit_scoring(probs, y[test_index], X.shape[1])
  profit /= n_folds
  return profit

In [ ]:
xgb_space_early_stop={
        'max_depth': hp.quniform("max_depth", 1, 11, 1),
        'gamma': hp.uniform ('gamma', 1,9),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'eta': hp.uniform('eta', 0.01, 0.3),
        'n_estimators': 500,
        'eval_metric': hp.choice('eval_metric', ['auc', 'error'])
    }

In [ ]:
xgb_space={
        'max_depth': hp.quniform("max_depth", 1, 11, 1),
        'gamma': hp.uniform ('gamma', 1,9),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'eta': hp.uniform('eta', 0.01, 0.3),
        'n_estimators': hp.quniform("n_estimators", 10, 300, 5)
    }


In [ ]:
def xgb_objective_early_stop(space):

    ### feature selection


    clf=xgb.XGBClassifier(
                    n_estimators=space['n_estimators'], max_depth = int(space['max_depth']), gamma = space['gamma'],
                    reg_lambda = space['reg_lambda'],min_child_weight=int(space['min_child_weight']),
                    colsample_bytree=space['colsample_bytree'], early_stopping_rounds=15, eta=space['eta'], eval_metric=space['eval_metric'], seed=123)

    #evaluation = [( Xtrain, ytrain), ( Xtest, ytest)]

    #clf.fit(X_boruta_forest, y, verbose=False)


    # pred = clf.predict(Xtest)
    # probs = clf.predict_proba(Xtest)[:,1]
    # accuracy = accuracy_score(ytest, pred)
    # p = Xtrain.shape[1]
    # money = profit_scoring(probs, ytest,p) #space["max_features"])
    money = xgb_cv_profit_early_stop(clf, X_boruta_forest, y)
    # print("Accuracy:", accuracy)
    # print("N features:",p) #space["max_features"])
    #print("Money:", money)
    return {'loss': -money, 'status': STATUS_OK }

In [ ]:
def xgb_objective(space):

    ### feature selection


    clf=xgb.XGBClassifier(
                    n_estimators=int(space['n_estimators']), max_depth = int(space['max_depth']), gamma = space['gamma'],
                    reg_lambda = space['reg_lambda'], min_child_weight=int(space['min_child_weight']),
                    colsample_bytree=space['colsample_bytree'], eta=space['eta'], seed=123)

    #evaluation = [( Xtrain, ytrain), ( Xtest, ytest)]

    #clf.fit(X_boruta_forest, y, verbose=False)


    # pred = clf.predict(Xtest)
    # probs = clf.predict_proba(Xtest)[:,1]
    # accuracy = accuracy_score(ytest, pred)
    # p = Xtrain.shape[1]
    # money = profit_scoring(probs, ytest,p) #space["max_features"])
    money = xgb_cv_profit(clf, X_boruta_forest, y)
    # print("Accuracy:", accuracy)
    # print("N features:",p) #space["max_features"])
    #print("Money:", money)
    return {'loss': -money, 'status': STATUS_OK }

## early stop

In [ ]:
# forest_selector = SelectFromModel(RandomForestClassifier(max_depth=7, random_state=123), threshold=-np.inf)
# forest_selector.fit(X_boruta, y)
A = [100, 102, 105]
B = [100, 102, 103, 105]
best_hypers_early_stop = []
trials_list_early_stop = []

for features in [A, B]:
#for i in range(3, 5):
  #forest_selector.set_params(max_features=i)
  #print(forest_selector.get_support(indices=True))
  X_boruta_forest = X[:, features]
  #X_boruta_forest = forest_selector.transform(X_boruta)
  #Xtrain, Xtest, ytrain, ytest = train_test_split(X_boruta_forest, y, test_size=0.2)
  trials = Trials()

  best_hyperparams = fmin(fn = xgb_objective_early_stop,
                        space = xgb_space_early_stop,
                        algo = tpe.suggest,
                        max_evals = 1000,
                        trials = trials)
  trials_list_early_stop.append(trials)
  best_hypers_early_stop.append(best_hyperparams)


100%|██████████| 1000/1000 [07:55<00:00,  2.10trial/s, best loss: -1360.0]


## no early stop

In [ ]:
# forest_selector = SelectFromModel(RandomForestClassifier(max_depth=7, random_state=123), threshold=-np.inf)
# forest_selector.fit(X_boruta, y)

A = [100, 102, 105]
B = [100, 102, 103, 105]

best_hypers = []
trials_list = []

for features in [A, B]:
#for i in range(3, 5):
  #forest_selector.set_params(max_features=i)
  #print(forest_selector.get_support(indices=True))
  #X_boruta_forest = forest_selector.transform(X_boruta)
  X_boruta_forest = X[:, features]
  #Xtrain, Xtest, ytrain, ytest = train_test_split(X_boruta_forest, y, test_size=0.2)
  trials = Trials()
  best_hyperparams = fmin(fn = xgb_objective,
                        space = xgb_space,
                        algo = tpe.suggest,
                        max_evals = 1000,
                        trials = trials)

  best_hypers.append(best_hyperparams)
  trials_list.append(trials)

100%|██████████| 1000/1000 [10:03<00:00,  1.66trial/s, best loss: -1370.0]


In [ ]:
import pickle

with open('trials_list_early_stop.pkl', 'wb') as f:
    pickle.dump(trials_list_early_stop, f)

with open('best_hypers_early_stop.pkl', 'wb') as f:
    pickle.dump(best_hypers_early_stop, f)

!cp trials_list_early_stop.pkl /content/drive/MyDrive/Studia/DS/Sem2/AML/Project2/Boosting/xgb_trials_early_stop.pkl
!cp best_hypers_early_stop.pkl /content/drive/MyDrive/Studia/DS/Sem2/AML/Project2/Boosting/xgb_best_hypers_early_stop.pkl

with open('trials_list.pkl', 'wb') as f:
    pickle.dump(trials_list, f)

with open('best_hypers.pkl', 'wb') as f:
    pickle.dump(best_hypers, f)

!cp trials_list.pkl /content/drive/MyDrive/Studia/DS/Sem2/AML/Project2/Boosting/xgb_trials.pkl
!cp best_hypers.pkl /content/drive/MyDrive/Studia/DS/Sem2/AML/Project2/Boosting/xgb_best_hypers.pkl

# LightGBM
Tunning hyperparameters with hyperopt

## early stopping

In [ ]:
def lgbm_cv_profit_early_stop(estimator, X, y, n_folds=5):
  skf = StratifiedKFold(n_folds)
  profit = 0
  for cv_idx, (train_index, test_index) in enumerate(skf.split(X, y)):
    Xtrain, Xval, ytrain, yval = train_test_split(X[train_index, :], y[train_index], test_size=0.2, stratify=y[train_index])
    evaluation = [(Xval, yval)]
    estimator.fit(X[train_index, :], y[train_index], eval_set=evaluation)#, verbose=False)
    print("Number of iterations:", estimator.best_iteration_)
    probs = estimator.predict_proba(X[test_index, :])[:,1]
    profit += profit_scoring(probs, y[test_index], X.shape[1])
  profit /= n_folds
  return profit

In [ ]:
lgbm_space_early_stop={
        'num_leaves': hp.quniform('num_leaves', 2, 128, 2),
        'lambda_l2' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'learning_rate': hp.uniform('learning_rate', 0.01, 0.3),
        'n_estimators': 500,
        'eval_metric': hp.choice('eval_metric', ['auc', 'error'])
    }

In [ ]:
def lgbm_objective_early_stop(space):

    ### feature selection


    clf=lgbm.LGBMClassifier(
                    n_estimators=space['n_estimators'], num_leaves = int(space['num_leaves']),
                    lambda_l2 = space['lambda_l2'],min_child_weight=int(space['min_child_weight']),
                    colsample_bytree=space['colsample_bytree'], early_stopping_rounds=15, learning_rate=space['learning_rate'], eval_metric=space['eval_metric'], seed=123, verbosity=-1)

    #evaluation = [( Xtrain, ytrain), ( Xtest, ytest)]

    #clf.fit(X_boruta_forest, y, verbose=False)


    # pred = clf.predict(Xtest)
    # probs = clf.predict_proba(Xtest)[:,1]
    # accuracy = accuracy_score(ytest, pred)
    # p = Xtrain.shape[1]
    # money = profit_scoring(probs, ytest,p) #space["max_features"])
    money = lgbm_cv_profit_early_stop(clf, X_boruta_forest, y)
    # print("Accuracy:", accuracy)
    # print("N features:",p) #space["max_features"])
    #print("Money:", money)
    return {'loss': -money, 'status': STATUS_OK }

In [ ]:
# forest_selector = SelectFromModel(RandomForestClassifier(max_depth=7), threshold=-np.inf)
# forest_selector.fit(X_boruta, y)
A = [100, 102, 105]
B = [100, 102, 103, 105]
lgbm_best_hypers_early_stop = []
lgbm_trials_early_stop = []

for features in [A,B]:
#for i in range(1, 8):
  #forest_selector.set_params(max_features=i)
  #X_boruta_forest = forest_selector.transform(X_boruta)
  X_boruta_forest = X[:, features]
  #Xtrain, Xtest, ytrain, ytest = train_test_split(X_boruta_forest, y, test_size=0.2)
  trials = Trials()
  best_hyperparams = fmin(fn = lgbm_objective_early_stop,
                        space = lgbm_space_early_stop,
                        algo = tpe.suggest,
                        max_evals = 5,
                        trials = trials)

  lgbm_best_hypers_early_stop.append(best_hyperparams)
  lgbm_trials_early_stop.append(trials)

## no early stopping

In [ ]:
def lgbm_cv_profit(estimator, X, y, n_folds=5):
  skf = StratifiedKFold(n_folds)
  profit = 0
  for cv_idx, (train_index, test_index) in enumerate(skf.split(X, y)):
    estimator.fit(X[train_index, :], y[train_index])#, verbose=False)
    probs = estimator.predict_proba(X[test_index, :])[:,1]
    profit += profit_scoring(probs, y[test_index], X.shape[1])
  profit /= n_folds
  return profit

In [ ]:
lgbm_space={
        'num_leaves': hp.quniform('num_leaves', 2, 128, 2),
        'lambda_l2' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'learning_rate': hp.uniform('learning_rate', 0.01, 0.3),
        'n_estimators': hp.quniform("n_estimators", 10, 300, 5)
    }

In [ ]:
def lgbm_objective(space):

    ### feature selection


    clf=lgbm.LGBMClassifier(
                    n_estimators=int(space['n_estimators']), num_leaves = int(space['num_leaves']),
                    lambda_l2 = space['lambda_l2'],min_child_weight=int(space['min_child_weight']),
                    colsample_bytree=space['colsample_bytree'], learning_rate=space['learning_rate'], seed=123, verbosity=-1)

    #evaluation = [( Xtrain, ytrain), ( Xtest, ytest)]

    #clf.fit(X_boruta_forest, y, verbose=False)


    # pred = clf.predict(Xtest)
    # probs = clf.predict_proba(Xtest)[:,1]
    # accuracy = accuracy_score(ytest, pred)
    # p = Xtrain.shape[1]
    # money = profit_scoring(probs, ytest,p) #space["max_features"])
    money = lgbm_cv_profit(clf, X_boruta_forest, y)
    # print("Accuracy:", accuracy)
    # print("N features:",p) #space["max_features"])
    #print("Money:", money)
    return {'loss': -money, 'status': STATUS_OK }

In [ ]:
A = [100, 102, 105]
B = [100, 102, 103, 105]
lgbm_best_hypers = []
lgbm_trials = []

for features in [A,B]:
#for i in range(1, 8):
  #forest_selector.set_params(max_features=i)
  #X_boruta_forest = forest_selector.transform(X_boruta)
  X_boruta_forest = X[:, features]
  #Xtrain, Xtest, ytrain, ytest = train_test_split(X_boruta_forest, y, test_size=0.2)
  trials = Trials()
  best_hyperparams = fmin(fn = lgbm_objective,
                        space = lgbm_space,
                        algo = tpe.suggest,
                        max_evals = 1000,
                        trials = trials)

  lgbm_best_hypers.append(best_hyperparams)
  lgbm_trials.append(trials)






with open('lgbm_trials.pkl', 'wb') as f:
    pickle.dump(lgbm_trials, f)

with open('lgbm_best_hypers.pkl', 'wb') as f:
    pickle.dump(lgbm_best_hypers, f)

!cp lgbm_trials.pkl /content/drive/MyDrive/Studia/DS/Sem2/AML/Project2/Boosting/lgbm_trials.pkl
!cp lgbm_best_hypers.pkl /content/drive/MyDrive/Studia/DS/Sem2/AML/Project2/Boosting/lgbm_best_hypers.pkl

100%|██████████| 1000/1000 [17:20<00:00,  1.04s/trial, best loss: -1376.0]
